In [ ]:
import tensorflow as tf
import sklearn
import pandas as pd
import numpy as np
import torch

In [2]:
!pip install tensorflow

     |████████████████████████████████| 228.5 MB 87 kB/s  eta 0:00:016    |████████████▍                   | 88.5 MB 1.4 MB/s eta 0:01:40     |████████████████████████████    | 200.2 MB 7.7 MB/s eta 0:00:04     |█████████████████████████████▊  | 212.0 MB 5.1 MB/s eta 0:00:04
     |████████████████████████████████| 65 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 13.2 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 463 kB/s eta 0:00:01
     |████████████████████████████████| 438 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 3.4 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 6.5 MB/s  eta 0:00:01
     |████████████████████████████████| 3.5 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 781

In [2]:
X_cols = [
    "B_2", "B_7", "B_18", "B_23", "B_32", "D_48",
    "D_55", "D_61", "D_121", "P_2", "S_11",
]

In [ ]:
chunksize = 1000000

train_df_iter = pd.read_csv("train_data.csv", chunksize=chunksize, usecols=["customer_ID"] + X_cols)


train_df = pd.DataFrame()
for i_chunk, chunk in enumerate(train_df_iter):
    train_df = pd.concat([train_df, chunk])
    print(train_df.shape)

In [5]:
train_labels_df = pd.read_csv("train_labels.csv")

In [6]:
train_df_mean = train_df.groupby("customer_ID")[X_cols].mean().reset_index()
train_df_last = train_df.groupby("customer_ID")[X_cols].last().reset_index()

train_df = pd.merge(
    left=train_df_mean, 
    right=train_df_last, 
    how="inner",
    on="customer_ID",
    suffixes=("_mean", "_last"),
)

train_df = pd.merge(train_df, train_labels_df, on="customer_ID", how="left")

In [7]:
train_df.head()

,customer_ID,B_2_mean,B_7_mean,B_18_mean,B_23_mean,B_32_mean,D_48_mean,D_55_mean,D_61_mean,D_121_mean,...,B_18_last,B_23_last,B_32_last,D_48_last,D_55_last,D_61_last,D_121_last,P_2_last,S_11_last,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.005086,0.036624,0.842565,0.026177,0.005084,0.240978,0.224432,0.225847,0.711829,...,1.007897,0.040367,0.007645,0.192376,0.187285,0.227637,0.719791,0.934745,0.402246,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.991083,0.028049,1.004884,0.013286,0.005960,0.048203,0.048069,0.053319,0.535892,...,1.003602,0.014705,0.008645,0.014696,0.036112,0.048978,0.551341,0.880519,0.363754,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.815677,0.034433,0.933173,0.023436,0.005220,0.092284,0.077362,0.109526,0.431903,...,1.004080,0.020228,0.001252,0.080370,0.098963,0.137834,0.444615,0.880875,0.280417,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.955264,0.062130,0.910999,0.046339,0.005252,0.076686,0.061726,0.066872,0.621386,...,1.007289,0.005060,0.007541,0.013057,0.021400,0.026844,0.629147,0.621776,0.368774,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.814543,0.115290,0.610194,0.100755,0.005332,0.253697,0.203298,0.356445,0.550940,...,0.531486,0.145214,0.007263,0.325121,0.254067,0.600739,0.565815,0.871900,0.326776,0


In [8]:
train_df.isna().sum()

customer_ID        0
B_2_mean          31
B_7_mean           0
B_18_mean          0
B_23_mean          0
B_32_mean          0
D_48_mean      28816
D_55_mean       2478
D_61_mean      21083
D_121_mean      4739
P_2_mean        2434
S_11_mean          0
B_2_last          31
B_7_last           0
B_18_last          0
B_23_last          0
B_32_last          0
D_48_last      28816
D_55_last       2478
D_61_last      21083
D_121_last      4739
P_2_last        2434
S_11_last          0
target             0
dtype: int64

In [9]:
train_df["B_2_mean"] = train_df["B_2_mean"].fillna(train_df["B_2_mean"].mean())
train_df["B_2_last"] = train_df["B_2_last"].fillna(train_df["B_2_last"].mean())

train_df["D_48_mean"] = train_df["D_48_mean"].fillna(train_df["D_48_mean"].mean())
train_df["D_48_last"] = train_df["D_48_last"].fillna(train_df["D_48_last"].mean())

train_df["D_55_mean"] = train_df["D_55_mean"].fillna(train_df["D_55_mean"].mean())
train_df["D_55_last"] = train_df["D_55_last"].fillna(train_df["D_55_last"].mean())

train_df["D_61_mean"] = train_df["D_61_mean"].fillna(train_df["D_61_mean"].mean())
train_df["D_61_last"] = train_df["D_61_last"].fillna(train_df["D_61_last"].mean())

train_df["D_121_mean"] = train_df["D_121_mean"].fillna(train_df["D_121_mean"].mean())
train_df["D_121_last"] = train_df["D_121_last"].fillna(train_df["D_121_last"].mean())

train_df["P_2_mean"] = train_df["P_2_mean"].fillna(train_df["P_2_mean"].mean())
train_df["P_2_last"] = train_df["P_2_last"].fillna(train_df["P_2_last"].mean())

In [10]:
_X_cols = train_df.columns[1:-1]

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df[_X_cols], train_df["target"], test_size=0.2, 
                                                    random_state=42, stratify=train_df["target"])

In [12]:
X_train = tf.expand_dims(X_train,axis = -1)
y_train = tf.expand_dims(y_train,axis = -1)
X_test = tf.expand_dims(X_test,axis = -1)
y_test = tf.expand_dims(y_test,axis = -1)

In [13]:
X_train.shape

TensorShape([367130, 22, 1])

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Input((22,1)),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, activation = "relu"),
    tf.keras.layers.Conv1D(filters = 128, kernel_size = 4, activation = "relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, activation = "relu"),
    tf.keras.layers.Conv1D(filters = 32, kernel_size = 2, activation = "relu"),
    tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(64, activation = "relu"),
    tf.keras.layers.Dense(16, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [15]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = "accuracy")

In [16]:
model.fit(X_train, y_train, epochs = 3)

Epoch 1/3
11473/11473 [==============================] - 125s 11ms/step - loss: 0.2992 - accuracy: 0.8624
Epoch 2/3
11473/11473 [==============================] - 119s 10ms/step - loss: 0.2889 - accuracy: 0.8669
Epoch 3/3
11473/11473 [==============================] - 121s 11ms/step - loss: 0.2860 - accuracy: 0.8682


In [17]:
predictions = model.predict(X_test)

2869/2869 [==============================] - 9s 3ms/step


In [18]:
model.evaluate(X_test, y_test)

2869/2869 [==============================] - 10s 3ms/step - loss: 0.2813 - accuracy: 0.8715


[0.28133344650268555, 0.8714903593063354]

In [78]:
chunksize = 1000000

test_df_iter = pd.read_csv("test_data.csv", chunksize=chunksize, usecols=["customer_ID"] + X_cols)

In [79]:
_index = []
_vals = []

for chunk in test_df_iter:
    _chunk_mean = chunk.groupby("customer_ID")[X_cols].mean().reset_index()
    _chunk_last = chunk.groupby("customer_ID")[X_cols].last().reset_index()
    _chunk = pd.merge(
        left=_chunk_mean, 
        right=_chunk_last, 
        how="inner",
        on="customer_ID",
        suffixes=("_mean", "_last"),
    )

    X_test = _chunk[_X_cols]
    X_test = X_test.fillna(0)
    X_test = tf.expand_dims(X_test, axis = -1)
    
    y_test_pred = model.predict(X_test)#:, 1]
    _index.extend(_chunk["customer_ID"])
    _vals.extend(y_test_pred)
    
    print(len(_index))

2543/2543 [==============================] - 8s 3ms/step
81358
2540/2540 [==============================] - 9s 4ms/step
162618
2546/2546 [==============================] - 10s 4ms/step
244059
2544/2544 [==============================] - 9s 3ms/step
325450
2543/2543 [==============================] - 9s 4ms/step
406815
2546/2546 [==============================] - 9s 4ms/step
488260
2541/2541 [==============================] - 11s 4ms/step
569555
2543/2543 [==============================] - 9s 4ms/step
650904
2542/2542 [==============================] - 9s 3ms/step
732217
2542/2542 [==============================] - 12s 5ms/step
813543
2547/2547 [==============================] - 12s 5ms/step
895040
925/925 [==============================] - 4s 5ms/step
924631


In [80]:
res_df = pd.DataFrame(
    {"customer_ID": _index, "prediction": np.squeeze(_vals)}
).groupby("customer_ID")["prediction"].mean().reset_index()

In [81]:
res_df.isna().sum()

customer_ID    0
prediction     0
dtype: int64

In [82]:
res_df.to_csv("tensorflow_predictions_submission3.csv", index = False)